In [1]:
import gensim
gensim.__version__

'2.2.0'

Visualization_TSNE_MDS_LLE_ISOMAP.ipynb에서 단어를 custom_tokenizer를 수행한 이후, 문서 벡터로 표현하여 2차원 벡터로 학습하였음

이 경우는 노이즈한 결과가 나오이기도 함. 시각화는 대상을 좋은 벡터로 표햔한 다음에 (representation learning), 이를 저차원의 벡터로 다시 한 번 압축함

대부분의 단어/문서 시각화는 Word2Vec, Doc2Vec, Glove, RNN, RBM 등으로 질 좋은 고차원 벡터의 representation을 얻은 뒤, 이를 다시 한 번 2차원 벡터로 압축하고 있음. 한 번에 풀리는 것은 존재하지 않음. 시각화와 같이 노이즈에 민감한 작업에서는 더더욱 그러한 경향이 있음

동일한 문서에 대하여 토크나이징을 하지 않은 체로 어절을 그대로 Word2Vec학습을 수행함. 데이터의 숫자가 정말로 풍부하고, 모델이 수많은 어절을 드대로 학습할 수 있을 만큼의 하드웨어면, 어절을 그대로 Word2Vec 학습하여도 그 결과는 충분히 쓸만하지만, 목적이 다를 수 있음

In [2]:
corpus_fname= '/home/paulkim/workspace/python/Korean_NLP/data/corpus_10days/news/2016-10-24_article_all_normed.txt'
word2vec_fname = '/home/paulkim/workspace/python/Korean_NLP/data/corpus_10days/models/2016-10-24_news_word2vec_model.pkl'

TRAIN_WORD2VEC = True

from corpus import Corpus
corpus = Corpus(corpus_fname, iter_sent=True)

i = 0
for idx, text in enumerate(corpus):
    i+=1
    print("%d : %s\n"%(idx, text))
    if i == 6:  break

0 : 

1 : 의원 60명 내무장관에게 서한 철거 때 어린이 안전 신경 써야

2 : 보호자 없는 아동 난민 70명은 영국에 도착

3 : 22일 경찰에 돌 던지는 칼레 난민 연합뉴스

4 : 파리 런던 연합뉴스 박성진 황정우 특파원 프랑스 칼레 난민촌 철거를 이틀 앞둔 22일 현지시간 난민촌 주변에서 현지 경찰과 난민이 충돌했다고 현지 프랑스앵포가 보도했다

5 : 난민촌 철거에 반대하는 난민 50명가량은 경찰을 향해 유리병과 돌을 던졌으며 경찰은 연막탄을 쏘면서 이들을 해산했다



In [3]:
len(corpus)

213087

doublespace line spliter이기 때문에 Word2VecCorpus는 ''를 기준으로 doc을 sents로 나눈 뒤, 각 sent를 다시 한 번 split()하여 word list로 yield함

In [4]:
import os

class Word2VecCorpus:
    def __init__(self, fname):
        self.fname = fname
        if not os.path.isfile(fname):
            print('File not found : %s'%fname)
            
    def __iter__(self):
        with open(self.fname, encoding='utf-8') as f:
            for doc in f:
                for sent in doc.strip().split('  '):
                    sent = sent.strip()
                    if not sent:
                        continue
                    yield sent.split()
                    
word2vec_corpus = Word2VecCorpus(corpus_fname)

for num_sent, sent in enumerate(word2vec_corpus):
    if num_sent > 5: break
    print(sent)

['의원', '60명', '내무장관에게', '서한', '철거', '때', '어린이', '안전', '신경', '써야']
['보호자', '없는', '아동', '난민', '70명은', '영국에', '도착']
['22일', '경찰에', '돌', '던지는', '칼레', '난민', '연합뉴스']
['파리', '런던', '연합뉴스', '박성진', '황정우', '특파원', '프랑스', '칼레', '난민촌', '철거를', '이틀', '앞둔', '22일', '현지시간', '난민촌', '주변에서', '현지', '경찰과', '난민이', '충돌했다고', '현지', '프랑스앵포가', '보도했다']
['난민촌', '철거에', '반대하는', '난민', '50명가량은', '경찰을', '향해', '유리병과', '돌을', '던졌으며', '경찰은', '연막탄을', '쏘면서', '이들을', '해산했다']
['이', '과정에서', '별다른', '부상자는', '나오지', '않았다']


In [6]:
from gensim.models import Word2Vec
import pickle

# 전부 때려넣어서 end-to-end로 접근
if TRAIN_WORD2VEC:
    word2vec_model = Word2Vec(word2vec_corpus, min_count=30)
    with open(word2vec_fname, 'wb') as f:
        pickle.dump(word2vec_model, f)
        
else:
    with open(word2vec_fname, 'rb') as f:
        word2vec_model = pickle.load(f)

In [7]:
len(word2vec_model.wv.syn0)

18329

In [8]:
test_words = ['김무성', 
              '박근혜', 
              '문재인', 
              '국방부', 
              '정부', 
              '국정원', 
              '대통령', 
              '축구', 
              '외교', 
              '정책', 
              '군대', 
              '미국', 
              '일본', 
              '중국'
             ]

어절만 이용하여 학습했음에도 불구하고, 김무성의 유사어로 정치인들이 등장함을 볼 수 있음. 이는 뉴스 코퍼스에서는 정치인의 이름만 적고 띄어스기를 하는 경우도 충분하기 때문임

하지만 '김무성 -대표도'와 같은 유사 어절이 등장하는 것은, 뉴스에서 '김무성'으로만 적는 경우와 '김무성 대표도'로 적는 경우가 혼재되어 있어서, '대표도'의 문맥이 '김무성'의 문맥과 유사하였기 때문임

비슷한 의미로, 박근혜 - 박 (6251)의 경우에는 '박근혜-대통령', '박 대통령'이 번갈아가며 이용되었기 때문임

그러나 국정원의 경우에는 총 42번 나왔으며, '박근혜 대통령', '박 대통령'이 번갈아가며 이용되었기 때문임.

그러나, 국정원의 경우에는 총 42번 나왔으며, min_count=30으로 하였기 때문에 18000여개의 단어 중에서는 infrequent 한 편에 속함. 그리고 이 때의 유사 어절들은 문맥상 잘 어울리지 않는 어절임

    Seed word = 국정원
    Vocab(count:42, index:12977, sample_int:4294967296)
    국정원 - 크리스 (33) (0.733)
    국정원 - 자격으로 (34) (0.716)
    국정원 - 남편인 (34) (0.714)
    국정원 - 출간한 (36) (0.708)
    국정원 - 단국대 (33) (0.707)
    국정원 - 준비는 (35) (0.707)
    국정원 - 대우학원 (34) (0.705)
    국정원 - 최경희 (32) (0.702)
    국정원 - 정동구 (31) (0.697)
    국정원 - 인터뷰가 (36) (0.696)
    
Word2Vec에서는 infrequent한 단어(어절)들의 유사 단어(어절)들은 infrequent한 경향이 있음. **빈도수가 충분하지 않을 경우**에는 학습이 잘 되지 않는 것으로 해석할 수 있음

어절을 그대로 학습할 경우에는, 아래와 같이 대통령의 유사어절로 '대통령 + 조사'의 어절들이 자주 등장함

    Seed word = 대통령
    Vocab(count:3411, index:89, sample_int:4294967296)
    대통령 - 대통령의 (3074) (0.827)
    대통령 - 대통령을 (287) (0.780)
    대통령 - 대통령과 (437) (0.768)
    대통령 - 대통령도 (125) (0.746)
    대통령 - 대통령이 (5054) (0.742)
    대통령 - 대통령에 (255) (0.725)
    대통령 - 대통령은 (2852) (0.720)
    대통령 - 대통령에게 (203) (0.711)
    대통령 - 대통령이다 (54) (0.678)
    대통령 - 대통령께서 (280) (0.672)
    

목적이 **원하는 Language model을 학습하는 것**이며, 이 결과는 **유용**함. 

하지만, **문맥적으로 유사한 단어들을 찾기 위해 Word2Vec을 학습하는 것이 목적**이면, 어절을 그대로 학습하는 것보다 **토크나이징**을 한 뒤, 이를 학습하여 이용해야 함

In [9]:
for word in test_words:
    if  (word in word2vec_model.wv.vocab) == False:
        continue
        
    print('\n\nSeed word = %s'%word)
    print(word2vec_model.wv.vocab[word])
    for similar_word, sim in word2vec_model.most_similar(word):
        print('%s - %s (%d) (%.3f)'%(word, similar_word, word2vec_model.wv.vocab[similar_word].count, sim))



Seed word = 김무성
Vocab(count:332, index:1414, sample_int:4294967296)
김무성 - 김종인 (165) (0.937)
김무성 - 유승민 (160) (0.891)
김무성 - 안철수 (345) (0.891)
김무성 - 손학규 (270) (0.883)
김무성 - 오세훈 (72) (0.881)
김무성 - 최경환 (43) (0.877)
김무성 - 대표도 (106) (0.867)
김무성 - 김부겸 (100) (0.844)
김무성 - 강석호 (43) (0.843)
김무성 - 개헌론자인 (36) (0.828)


Seed word = 박근혜
Vocab(count:4420, index:62, sample_int:4294967296)
박근혜 - 이승만 (36) (0.747)
박근혜 - 박정희 (216) (0.723)
박근혜 - 마두로 (46) (0.720)
박근혜 - 박 (6251) (0.719)
박근혜 - 뒤늦은 (30) (0.711)
박근혜 - 국정과 (37) (0.698)
박근혜 - 두테르테 (218) (0.679)
박근혜 - 민생에 (38) (0.674)
박근혜 - 전두환 (44) (0.668)
박근혜 - 오바마 (209) (0.663)


Seed word = 문재인
Vocab(count:940, index:392, sample_int:4294967296)
문재인 - 손학규 (270) (0.869)
문재인 - 김종인 (165) (0.863)
문재인 - 안철수 (345) (0.863)
문재인 - 비서실장이 (97) (0.851)
문재인 - 문 (932) (0.851)
문재인 - 장관의 (162) (0.834)
문재인 - 송민순 (663) (0.807)
문재인 - 김무성 (332) (0.795)
문재인 - 유승민 (160) (0.785)
문재인 - 안보실장이 (81) (0.781)


Seed word = 국방부
Vocab(count:247, index:2003, sample_int:4294967296)
국방부 - 외교부 